In [53]:
import gzip
from tqdm import tqdm
import json
from random import shuffle

ret = {
    '[music] ': [],
    '[movie] ': [],
    '[book] ': [],
    '[game] ': [],
}

with gzip.open('/mnt/ceph/storage/data-in-progress/data-research/web-search/false-memories/reddit-tomt/tomt-dataset-26-01-2023/reddit-tomt-submissions.jsonl.gz', 'r') as input_file:
    for l in tqdm(input_file):
        l = json.loads(l)
        for k in ret.keys():
            if k in l['title'].lower():
                ret[k] += [l]


1279425it [06:02, 3534.19it/s]


In [19]:
for k in ret.keys():
    entries = [i for i in ret[k]]
    shuffle(entries)
    entries = entries[:10000]
    
    print(len(entries))
    ret[k] = {
        #'test': entries[:100],
        #'validation': entries[100:200],
        'training': entries[200:],
    }

10000
10000
10000
10000


In [36]:
def transform(i):
    try:
        text = (i['title'].split(']')[-1].strip()) + ' ' + i['content'].strip()
        time_to_solve = None if not i['solved_utc'] else int(i['solved_utc']) - int(i['created_utc'])
    
        return {'text': text, 'time_to_solve': time_to_solve}
    except:
        return None

In [50]:
qid = 1
for data_set in ['music', 'movie', 'book', 'game']:
    for data_type in ['training']:
        with open(f'/mnt/ceph/storage/data-in-progress/data-research/web-search/false-memories/reddit-tomt/tomt-dataset-26-01-2023//qpp-sample/{data_set}-{data_type}.jsonl', 'w') as outp_f:
            for l in ret[f'[{data_set}] '][data_type]:
                l = transform(l)
                if not l:
                    continue
                l['qid'] = qid
                qid += 1
                outp_f.write(json.dumps(l) + '\n')


In [52]:
len(ret[f'[movie] ']['training'])

127

In [46]:


all_test = ret['[music] ']['test'][:25] + ret['[movie] ']['test'][:25] + ret['[book] ']['test'][:25] + ret['[game] ']['test'][:25]

In [49]:
!sync

In [48]:
with open(f'/mnt/ceph/storage/data-in-progress/data-research/web-search/false-memories/reddit-tomt/tomt-dataset-26-01-2023//qpp-sample/all-categories-test.jsonl', 'w') as outp_f:
    for l in all_test:
        l = transform(l)
        if not l:
            continue
        l['qid'] = qid
        qid += 1
        outp_f.write(json.dumps(l) + '\n')


# Create query file

In [44]:
directory = '/mnt/ceph/storage/data-in-progress/data-research/web-search/false-memories/reddit-tomt/tomt-dataset-26-01-2023//qpp-sample/'

with open(f'{directory}/qpptk-queries', 'w') as output_file:
    for data_set in ['music', 'movie', 'book', 'game']:
        with open(f'{directory}{data_set}-test.jsonl', 'r') as input_file:
            for l in input_file:
                l = json.loads(l)
                output_file.write(str(l['qid']) + ' ' + (' '.join(l['text'].split())).strip() + '\n')
        